In [9]:
import secrets
import time

# **Criptografía RSA: De la Matemática al Código**


Este notebook explica el funcionamiento del algoritmo RSA, desde sus cimientos matemáticos hasta la implementación de un ataque de factorización de Fermat.

## 1. Fundamentos Matemáticos

Antes de implementar RSA, necesitamos algunas herramientas de teoría de números.

**1.1 Exponenciación Modular Rápida.**

Uno de los pilares más importantes del RSA consiste en elevar un mensaje a una potencia muy grande: $a^b \pmod n$. En un entorno real, el exponente $b$ puede tener cientos de dígitos, lo que hace que el cálculo directo sea imposible por dos razones:
1. *Memoria*: El número resultante tendría trillones de dígitos.
2. *Tiempo*: Realizar trillones de multiplicaciones tomaría años. 

La Estrategia: "Square-and-Multiply" en lugar de multiplicar la base por sí misma linealmente, aprovechamos la representación binaria del exponente. Esto reduce la complejidad de $O(b)$ a $O(\log b)$.

El algoritmo de exponenciación binaria se basa en propiedad distributiva de la aritmética modular: $(a \cdot b) \pmod n = [(a \pmod n) \cdot (b \pmod n)] \pmod n$.

Y se apoya en tres pilares:
1. **Descomposición Binaria:** Todo exponente se expresa como suma de potencias de 2. Por ejemplo, si $b=13$ $(1101)_2$, calculamos $a^{13} = a^8 \cdot a^4 \cdot a^1$.
2. **Cuadrados Sucesivos:** Obtenemos potencias grandes elevando al cuadrado el resultado anterior ($a \to a^2 \to a^4 \to a^8$). Esto requiere solo $\log b$ operaciones.
3. **Aritmética Modular:** Aplicamos el módulo en cada multiplicación intermedia. Esto mantiene los números siempre menores a $n$, evitando el overflow de memoria.

In [10]:
def exponenciacion_modular(a, b, n):
    resultado = 1
    a = a % n
    # El bucle continúa hasta procesar todos los bits del exponente.
    while b > 0:
        if b & 1:       # Si el bit actual es 1, se multiplica el resultado.
            resultado = (resultado * a) % n
        a = (a * a) % n # La base se eleva al cuadrado en cada paso.
        b >>= 1         # Desplazamiento de bit para procesar el exponente

    return resultado

**1.2 Algoritmo de Euclides Extendido e Inverso Modular**

En RSA, la clave pública ($n,e$) y la clave privada ($n,d$) están vinculadas matemáticamente. Para generar $d$, tenemos que resolver la siguiente congruencia:
$$e \cdot d \equiv 1 \pmod{\phi(n)}$$
donde, si $n = p \cdot q$ con $p$ y $q$ primos, se cumple que: $\phi(n) = (p - 1)(q - 1)$. Esta función, llamada función totiente de Euler, juega un rol fundamental en el teorema de Euler (generalización del pequeño teorema de Fermat), que garantiza la corrección del esquema RSA.

Para hallar $d$ de forma eficiente, transformamos la congruencia en una ecuación lineal conocida como Identidad de Bézout:
$$e \cdot d + \phi(n) \cdot k = \text{mcd}(e, \phi(n))$$
Como en RSA elegimos $e$ para que sea coprimo con $\phi(n)$, el $\text{mcd}$ es 1. El algoritmo encuentra $d$ mediante un proceso de sustitución hacia atrás.

In [11]:
def mcd_extendido(a, b):
    # Caso base: mcd(0, b) = b. La combinación es 0*a + 1*b = b
    if a == 0:
        return b, 0, 1
    
    # Obtenemos los coeficientes (x1, y1) del paso siguiente
    g, x1, y1 = mcd_extendido(b % a, a)
    
    # Aplicamos la sustitución para hallar los coeficientes actuales
    x = y1 - (b // a) * x1
    y = x1
    
    return g, x, y

def inverso_modular(a, m):
    g, x, _ = mcd_extendido(a, m)
    if g != 1:
        raise Exception("No existe inverso modular")
    # Aseguramos que el resultado sea positivo dentro del módulo m
    return x % m


## 2. Generación de Primos y el Test de Miller-Rabin

RSA depende de encontrar dos números primos gigantes ($p$ y $q$). Sin embargo, no existe una fórmula matemática para "fabricar" números primos. La solución en ingeniería es simple pero potente: generar un número aleatorio muy grande y probar si es primo.

**2.1 El Test Probabilístico de Miller-Rabin**

Dado que verificar la primalidad de forma absoluta (mediante división) es demasiado lento, usamos el test de Miller-Rabin. Este test no nos dice con exactitud si un número es primo, sino que nos asegura una alta probabilidad de que lo sea.

Se basa en el resultado siguiente:

**Proposición:** Sea $n > 2$ un número primo, y sea $n - 1 = 2^s \cdot d$ con $d$ impar. Para cualquier entero $a$ tal que $1 \leq a < n$, se cumple al menos una de estas condiciones:
1. $a^d \equiv 1 \pmod n$
2. $a^{2^r \cdot d} \equiv -1 \pmod n$ para algún $r$ tal que $0 \leq r < s$.

Si un número $n$ no cumple ninguna de estas condiciones para una base $a$ dada, entonces $n$ es compuesto. Si las cumple, decimos que $n$ es un "probable primo" y probamos con otra base $a$ para aumentar nuestra certeza.

La función ronda_miller_rabin es una implementación directa de esta proposición:

**Preparación**: Calculamos $s$ y $d$ dividiendo $n-1$ por 2 sucesivamente hasta que el resto sea impar (el primer while).
 - **Caso 1** ($a^d \equiv 1$): Calculamos x = exponenciacion_modular(a, d, n). Si el resultado es $1$ (o $n-1$, que es $-1$ en aritmética modular), el número pasa la ronda.
 - **Caso 2** ($a^{2^r \cdot d} \equiv -1$): Si el paso anterior no dio $1$, elevamos $x$ al cuadrado sucesivamente (x = (x * x) % n). Esto equivale a calcular $a^{2^1 \cdot d}, a^{2^2 \cdot d}, \dots$. Si en algún paso obtenemos $n-1$, el número pasa la ronda.

In [12]:
def ronda_miller_rabin(n, a):
    # n - 1 = d * 2^s con d impar.
    d = n - 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1
    
    x = exponenciacion_modular(a, d, n)
    if x == 1 or x == n - 1:
        return True
    for _ in range(0,s - 1,1):
        x = (x * x) % n
        if x == n - 1:
            return True
    return False

def es_primo_probable(n, rondas=40):
    if n < 2: return False
    if n in (2, 3): return True
    if n % 2 == 0: return False
    for _ in range(rondas):
        # Genera un número seguro en el rango [2, n-2]
        a = secrets.randbelow(n - 4) + 2 
        if not ronda_miller_rabin(n, a):
            return False
    return True

def generar_primo(bits):
    while True:
        n = secrets.randbits(bits)
        n |= (1 << (bits - 1)) | 1 
        if es_primo_probable(n):
            return n

## 3. El Algoritmo RSA

**3.1 Generación de Claves**

Es el proceso de crear el candado (clave pública) y su llave (clave privada).

1. **Selección de Primos:** Generamos dos primos grandes, $p$ y $q$, mediante el test de Miller-Rabin.
2. **Módulo ($n$):** Calculamos $n = p \cdot q$. Este valor es público y define la longitud de la clave.
3. **Función Totiente: $\phi(n) = (p-1)(q-1)$** Este valor debe ser secreto, pues es la pieza necesaria para derivar la clave privada.
4. **Exponente Público ($e$):** Se elige un valor estándar (típicamente $65537$) que sea coprimo con $\phi(n)$.
5. **Exponente Privado ($d$):** Se calcula como el inverso modular de $e \pmod{\phi(n)}$ usando el Algoritmo de Euclides Extendido.

**3.2 Cifrado y Descifrado**

- **Cifrar:** $C = M^e \pmod n$. El emisor transforma el mensaje $M$ en un criptograma $C$ usando la clave pública.
- **Descifrar:** $M = C^d \pmod n$. Solo el poseedor de la clave privada $d$ puede revertir la operación.

**3.3 ¿Por qué funciona?**

Debido al Teorema de Euler, al elevar un mensaje $M$ a la potencia $e$ (cifrar) y luego el resultado a la potencia $d$ (descifrar), regresamos matemáticamente al mensaje original:
$$C^d \equiv (M^e)^d \equiv M^{ed} \equiv M^1 \pmod n$$

Observar que el método sirve para abrir un canal de comunicación donde todo aquél que tenga la clave pública puede encriptar mensajes, pero sólo quién/es posee/n la clave privada puede/n descrifrarlo.

Intuición de Seguridad: La seguridad de RSA reside en que es fácil calcular $n = p \cdot q$, pero es computacionalmente imposible obtener $p$ y $q$ a partir de $n$ si estos son suficientemente grandes. Sin $p$ y $q$, no se puede calcular $\phi(n)$, y por ende, es imposible hallar la clave privada $d$.

In [13]:
def generar_claves_rsa(bits=1024):
    p = generar_primo(bits)
    q = generar_primo(bits)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = inverso_modular(e, phi)
    return (n, e), (n, d)

def cifrar_rsa(mensaje, clave_publica):
    n, e = clave_publica
    return exponenciacion_modular(mensaje, e, n)

def descifrar_rsa(cifrado, clave_privada):
    n, d = clave_privada
    return exponenciacion_modular(cifrado, d, n)

## 4. Demostración del Flujo

En RSA, los mensajes deben ser tratados como números; por ello, utilizaremos funciones auxiliares para transformar texto en enteros (codificación) y viceversa.

Este ejemplo simula el escenario real:

1. El Receptor prepara su "cerradura" y su "llave".
2. El Emisor transforma su mensaje en un número, lo cifra con la clave pública y lo envía por un canal inseguro.
3. El Receptor utiliza su clave privada para revertir el cifrado y recuperar el texto original.

In [14]:
# Funciones auxiliares de conversión texto-entero y viceversa.
def texto_a_entero(texto):
    data = texto.encode("utf-8")
    return int.from_bytes(data, "big")

def entero_a_texto(entero):
    data = entero.to_bytes((entero.bit_length() + 7) // 8, "big")
    return data.decode("utf-8")

# 1. Configuración del Receptor (Generación de Claves)
# 1024 bits para asegurar una complejidad computacional adecuada.
publica_rec, privada_rec = generar_claves_rsa(1024)

# 2. Acción del Emisor (Cifrado)
mensaje_original = "Información confidencial: El presupuesto ha sido aprobado."
print(f"Mensaje Original: {mensaje_original}")

msj_entero = texto_a_entero(mensaje_original)
mensaje_cifrado = cifrar_rsa(msj_entero, publica_rec)

print(f"\nMensaje cifrado enviado al canal: \n{mensaje_cifrado}")

# 3. Recepción y Descifrado
msj_final_entero = descifrar_rsa(mensaje_cifrado, privada_rec)
print(f"\nMensaje descifrado por el Receptor: {entero_a_texto(msj_final_entero)}")

Mensaje Original: Información confidencial: El presupuesto ha sido aprobado.

Mensaje cifrado enviado al canal: 
515523913788869661292267514863615975322808012472260316415914485533581734894297479011776930170425480351040159834877333893379749162964306667635639391561222180057624261184142226294377836548498810316909061190643091668609734703966196855823434461711485813827703670320670251658612426470615604452215361188408018064689595480030927845604886434007194249200714256877847213907403639949080085010573606148804901367792923797569861114631485082705982616765513357934258117221434347072855835904356200017129715407049644394742276742054914025748061682021569877043443364690298091180579241286800073427576225783022154318808152283665632156973

Mensaje descifrado por el Receptor: Información confidencial: El presupuesto ha sido aprobado.


## 5. Vulnerabilidades: El Ataque de Fermat

La seguridad de RSA no solo depende de que $p$ y $q$ sean primos, sino también de cómo se eligen. Si los primos están demasiado cerca uno del otro, el módulo $n$ puede romperse en segundos.

El ataque de Fermat se basa en que cualquier número impar puede representarse como una diferencia de cuadrados:
$$n = x^2 - y^2 = (x - y)(x + y)$$
Si definimos los factores como $p = (x - y)$ y $q = (x + y)$, cuando $p \approx q$, la diferencia $y$ es muy pequeña. Esto implica que $x$ estará apenas por encima de $\sqrt{n}$.

**Estrategia del atacante:**
1. Empezar con $x = \lceil\sqrt{n}\rceil$.
2. Probar valores sucesivos de $x$ ($x+1, x+2, \dots$) hasta que $x^2 - n$ sea un cuadrado perfecto ($y^2$).
3. Una vez hallado, los factores se recuperan fácilmente como $x-y$ y $x+y$.

In [15]:
def raiz_entera(n):
    """Calcula la raíz cuadrada entera exacta."""
    if n == 0: return 0
    x = 2**(n.bit_length() // 2 + 1)
    while True:
        y = (x + n // x) // 2
        if y >= x: return x
        x = y

def ataque_de_fermat(n, K=10000000):
    """
    Intenta factorizar n si sus factores p y q están cerca.
    """
    x = raiz_entera(n)
    if x * x < n: x += 1
    for _ in range(K):
        y2 = x*x - n
        y = raiz_entera(y2)
        if y*y == y2:
            return (x - y), (x + y)
        x += 1
    return None

**Simulación de Intercepción y Ataque**

A continuación, simulamos un escenario de "Criptoanálisis". Forzaremos la creación de una clave débil (donde $p$ y $q$ son casi iguales) para demostrar cómo un atacante puede recuperar la clave privada y leer un mensaje confidencial en milisegundos.

In [16]:
# 1. Generación de clave vulnerable (Primos cercanos)
p_vulnerable = generar_primo(1024)
q_vulnerable = p_vulnerable + 2  # Se fuerza un segundo primo muy cercano al primero
while not es_primo_probable(q_vulnerable):
    q_vulnerable += 2

n_vulnerable = p_vulnerable * q_vulnerable
e = 65537
publica_vulnerable = (n_vulnerable, e)

# 2. El Emisor envía un mensaje sin saber que la clave es débil
mensaje_secreto = "Clave de acceso a la base de datos: Admin123"
cifrado_interceptado = cifrar_rsa(texto_a_entero(mensaje_secreto), publica_vulnerable)

print(f"Paquete interceptado: {str(cifrado_interceptado)[:100]}...")

print("--- INICIO DE INTERCEPCIÓN ---")

# 3. El Interceptor ejecuta el ataque de Fermat
print("\n[Interceptor] Analizando vulnerabilidades en 'n'...")
inicio_ataque = time.time()

# El ataque busca x tal que x^2 - n sea un cuadrado perfecto
resultado = ataque_de_fermat(n_vulnerable, K=10**6)
fin_ataque = time.time()

if resultado:
    p_ext, q_ext = resultado
    print(f"[Interceptor] ¡Factorización exitosa!")
    print(f"Tiempo de quiebre: {fin_ataque - inicio_ataque:.4f} segundos")
    
    # Reconstrucción de la clave privada
    phi_ext = (p_ext - 1) * (q_ext - 1)
    d_ext = inverso_modular(e, phi_ext)
    privada_ext = (n_vulnerable, d_ext)
    
    # Descifrado del mensaje interceptado
    mensaje_recuperado = entero_a_texto(descifrar_rsa(cifrado_interceptado, privada_ext))
    print(f"[Interceptor] Contenido recuperado: '{mensaje_recuperado}'")
else:
    print("[Interceptor] El ataque falló. La clave no presenta la vulnerabilidad de Fermat.")


Paquete interceptado: 2986993425102405494364589584581385730498694413006028689364852756992679017044099311252213179804872868...
--- INICIO DE INTERCEPCIÓN ---

[Interceptor] Analizando vulnerabilidades en 'n'...
[Interceptor] ¡Factorización exitosa!
Tiempo de quiebre: 0.0002 segundos
[Interceptor] Contenido recuperado: 'Clave de acceso a la base de datos: Admin123'


## 6. Comentarios Finales

Este notebook fue desarrollado como un ejercicio de implementación y análisis, con foco en recorrer de forma explícita cada paso matemático involucrado en RSA y en una de sus vulnerabilidades clásicas. Las decisiones de diseño priorizan la claridad del razonamiento y la trazabilidad de los cálculos por sobre la concisión o el uso de abstracciones de alto nivel. Durante el desarrollo se contrastaron resultados y enfoques con referencias externas y herramientas de apoyo, pero cada componente fue implementado y verificado dentro del propio flujo del notebook. El contenido está pensado para ser leído, cuestionado y modificado, más que para ser ejecutado como una solución cerrada.